# Домашнее задание № 4. Языковые модели

## Задание 1 (8 баллов).

В семинаре для генерации мы использовали предположение маркова и считали, что слово зависит только от 1 предыдущего слова. Но ничто нам не мешает попробовать увеличить размер окна и учитывать два или даже три прошлых слова. Для них мы еще сможем собрать достаточно статистик и, логично предположить, что качество сгенерированного текста должно вырасти.

Попробуйте сделать языковую модель, которая будет учитывать два предыдущих слова при генерации текста.
Сгенерируйте несколько текстов (3-5) и расчитайте перплексию получившейся модели. 
Можно использовать данные из семинара или любые другие (сопоставимые или большие по объему). Перплексию рассчитывайте на 10-50 отложенных предложениях (они не должны использоваться при сборе статистик).


Подсказки:  
    - нужно будет добавить еще один тэг <start>  
    - еще одна матрица не нужна, можно по строкам хронить биграмы, а по колонкам униграммы  
    - тексты должны быть очень похожи на нормальные (если у вас получается рандомная каша, вы что-то делаете не так). 

In [1]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
from nltk.tokenize import sent_tokenize
import numpy as np
from collections import Counter
from scipy.sparse import csr_matrix, csc_matrix, lil_matrix

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\imphi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
dvach = open('2ch_corpus.txt', encoding = 'utf-8').read()

In [4]:
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text

In [5]:
norm_dvach = normalize(dvach)

In [7]:
vocab_dvach = Counter(norm_dvach)

In [9]:
probas_dvach = Counter({word:c/len(norm_dvach) for word, c in vocab_dvach.items()})

In [10]:
def ngrammer(tokens, n):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [11]:
sentences_dvach = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(dvach)]

In [12]:
unigrams_dvach = Counter()
bigrams_dvach = Counter()
trigrams_dvach = Counter()

for sentence in sentences_dvach:
    unigrams_dvach.update(sentence)
    bigrams_dvach.update(ngrammer(sentence, n=2))
    trigrams_dvach.update(ngrammer(sentence, n=3))
    

In [14]:
matrix_dvach = lil_matrix((len(bigrams_dvach), 
                   len(unigrams_dvach)))

id2word_dvach = list(unigrams_dvach)
word2id_dvach = {word:i for i, word in enumerate(id2word_dvach)}

id2bigram_dvach = list(bigrams_dvach)
bigram2id_dvach = {word:i for i, word in enumerate(id2bigram_dvach)}

for ngram in trigrams_dvach:
    word1, word2, word3 = ngram.split()
    bigram = word1+' '+word2 
    matrix_dvach[bigram2id_dvach[bigram], word2id_dvach[word3]] =  (trigrams_dvach[ngram]/
                                                                     bigrams_dvach[bigram])

In [15]:
matrix_dvach = csr_matrix(matrix_dvach)

In [16]:
def generate(matrix, id2bigram, id2word, bigram2id, n=100, start='<start> <start>'):
    text = []
    current_idx = bigram2id[start]
    for i in range(n):
        
        chosen = np.random.choice(list(range(matrix.shape[1])), p=matrix[current_idx].toarray()[0])
        nxt = id2word[chosen]
        text.append(nxt)
        
        bi = id2bigram[current_idx]
        prcd = bi.split()[1]
        
        if id2word[chosen] == '<end>':
            current_idx = bigram2id[start]
        else:
            prod = prcd + ' ' + nxt
            current_idx = bigram2id[prod]
    return ' '.join(text)

In [17]:
for k in range(5):
    generated = generate(matrix_dvach, id2bigram_dvach, id2word_dvach, bigram2id_dvach).replace('<end>', '')
    print(k+1, '\n', generated, '\n')

1 
 вроде бы отчасти на них влияет  вот это я сама власть и ассортимент завезли  мы даже предсказать не можем  наказание пусть они и увидят это то же что там она не нужна если тян онлайн возможно прямо сейчас возьму себя в божеский вид повернув до нужного угла  так я тоже на познакомился с тян а как это сделала мс так как ты их не два  кризис раннего возраста скромные  пиздос дожили модами теперь допиливают то что у меня у друзей путина и до этого  сан вынужден сотрудничать с вояками  мда походу анимация 

2 
 ты ничего не писал что прям никто больше не сделают миллиардеров счастливыми  судить людей по знакомству  они уже один раз цепануть телку и утащить ее в лоб немцетяжи 90 мм м 3 лучше бр-471 б у купи на которой он второй ебали тремя хуями и т д потому что моды пиратские эти безвольные долбоебы все удалят как totw оно и понятно интеграл от коцикла по циклу  на видеоролике было прекрасно  и так далее уже не помню такого за собой не замечаю или игнорю в в трете смотрем чирвоны смотре

## Задание № 2* (2 балла). 

Прочитайте главу про языковое моделирование в книге Журафски и Мартина - https://web.stanford.edu/~jurafsky/slp3/3.pdf

Развернуто (в пределах 1000 знаков) ответьте на вопросы (по-русски):

1. Что можно делать с проблемой несловарных слов? В семинаре мы просто использовали какое-то маленькое значение вероятности, а какие есть другие способы?

2. Что такое сглаживание (smoothing)?